In [2]:
#from azureml.opendatasets import NycTlcGreen
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import LabelEncoder   
from sklearn.model_selection import train_test_split

In [2]:
import azureml.core
from azureml.core import Workspace  
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.60


In [3]:
data=pd.read_csv('housing.csv')

In [4]:
data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
longitude             20640 non-null float64
latitude              20640 non-null float64
housing_median_age    20640 non-null float64
total_rooms           20640 non-null float64
total_bedrooms        20433 non-null float64
population            20640 non-null float64
households            20640 non-null float64
median_income         20640 non-null float64
median_house_value    20640 non-null float64
ocean_proximity       20640 non-null object
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [6]:
#Checking types of value in ocean proximity
data['ocean_proximity'].unique()

array(['NEAR BAY', '<1H OCEAN', 'INLAND', 'NEAR OCEAN', 'ISLAND'],
      dtype=object)

In [7]:
#Performing Lable Encoder 
labelencoder = LabelEncoder()
ocean_proximity= labelencoder.fit_transform(data['ocean_proximity'])

In [8]:
type(ocean_proximity)

numpy.ndarray

In [9]:
#dropping and joining the encode column
new_data=data.drop(labels='ocean_proximity',axis=1)
new_data = new_data.merge(pd.DataFrame(ocean_proximity), how='left', left_index=True, right_index=True)

In [10]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
longitude             20640 non-null float64
latitude              20640 non-null float64
housing_median_age    20640 non-null float64
total_rooms           20640 non-null float64
total_bedrooms        20433 non-null float64
population            20640 non-null float64
households            20640 non-null float64
median_income         20640 non-null float64
median_house_value    20640 non-null float64
0                     20640 non-null int32
dtypes: float64(9), int32(1)
memory usage: 1.5 MB


In [12]:
new_data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,0
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,3
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,3
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,3
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,3
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,3


In [13]:
new_data1= new_data.rename(columns = {0:"ocean_proximity"}) 

In [14]:
new_data1.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,3
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,3
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,3
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,3
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,3


In [15]:
#Creating X and y datasets
y=new_data1['median_house_value']
X=new_data1.drop('median_house_value',axis=1)

In [16]:
#Splitting the datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [17]:
import logging

automl_settings = {
    "iteration_timeout_minutes": 2,
    "iterations": 2,
    "primary_metric": 'spearman_correlation',
    "preprocess": True,
    "verbosity": logging.INFO,
    "n_cross_validations": 5
}

In [18]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(task='regression',
                             debug_log='automated_ml_errors.log',
                             X=X_train.values,
                             y=y_train.values.flatten(),
                             **automl_settings)

W0910 21:23:16.450878 12396 deprecation_wrapper.py:119] From C:\Users\hni7kor\AppData\Roaming\Python\Python37\site-packages\azureml\automl\core\_vendor\automl\client\core\common\tf_wrappers.py:36: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0910 21:23:16.451879 12396 deprecation_wrapper.py:119] From C:\Users\hni7kor\AppData\Roaming\Python\Python37\site-packages\azureml\automl\core\_vendor\automl\client\core\common\tf_wrappers.py:36: The name tf.logging.ERROR is deprecated. Please use tf.compat.v1.logging.ERROR instead.



In [19]:
from azureml.core.workspace import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication
interactive_auth = InteractiveLoginAuthentication()
ws = Workspace.from_config()

In [20]:
from azureml.core.experiment import Experiment
experiment = Experiment(ws, "House")
local_run = experiment.submit(automl_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_b6ad9b99-21ed-4cf0-9945-3d296ca2c7eb
Current status: DatasetFeaturization. Beginning to featurize the dataset.
Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturizationCompleted. Completed featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS SUMMARY:
For more details, use API: run.get_guardrails()

TYPE:         Missing Values Imputation
STATUS:       FIXED
DESCRIPTION:  The training data had the following missing values which were resolved.

Please review your data source for data quality issues and possibly filter out the rows with these missing values.

If the missing values are expected, you can either accept the above imputation, or implement

In [21]:
from azureml.widgets import RunDetails
RunDetails(local_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [22]:
best_run, fitted_model = local_run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: House,
Id: AutoML_b6ad9b99-21ed-4cf0-9945-3d296ca2c7eb_1,
Type: None,
Status: Completed)
RegressionPipeline(pipeline=Pipeline(memory=None,
     steps=[('datatransformer', DataTransformer(enable_feature_sweeping=None, feature_sweeping_timeout=None,
        is_onnx_compatible=None, logger=None, observer=None, task=None)), ('MinMaxScaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('RandomForestRegressor', RandomForestRegressor(bootstrap...timators=25, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False))]),
          stddev=None)


In [23]:
y_predict = fitted_model.predict(X_test.values)
print(y_predict[:10])

[ 59300.91176656  82506.38471386 360994.46302421 272110.51114882
 289259.72173085 166841.67035211 247159.12050413 182288.12320911
 276452.33107215 447783.49969856]


In [24]:
from sklearn.metrics import mean_squared_error
from math import sqrt

y_actual = y_test.values.flatten().tolist()
rmse = sqrt(mean_squared_error(y_actual, y_predict))
rmse

62340.57010125714

In [25]:
y_actual

[47700.0,
 45800.0,
 500001.0,
 218600.0,
 278000.0,
 158700.0,
 198200.0,
 157500.0,
 340000.0,
 446600.0,
 123200.0,
 253900.0,
 215100.0,
 220500.0,
 219800.0,
 136200.0,
 178400.0,
 187500.0,
 139800.0,
 137500.0,
 425000.0,
 405600.0,
 138800.0,
 500001.0,
 162100.0,
 51400.0,
 151300.0,
 94800.0,
 204100.0,
 361500.0,
 85100.0,
 138400.0,
 137500.0,
 465000.0,
 271800.0,
 239400.0,
 340900.0,
 150400.0,
 128100.0,
 156000.0,
 258100.0,
 90500.0,
 71300.0,
 219700.0,
 151000.0,
 290100.0,
 142000.0,
 291500.0,
 255400.0,
 98100.0,
 251900.0,
 162100.0,
 129400.0,
 89400.0,
 150500.0,
 133800.0,
 80400.0,
 211100.0,
 255600.0,
 214500.0,
 500001.0,
 202100.0,
 81500.0,
 98800.0,
 281900.0,
 184600.0,
 254700.0,
 143100.0,
 162500.0,
 142600.0,
 85900.0,
 186900.0,
 71300.0,
 174400.0,
 225000.0,
 146500.0,
 405000.0,
 189600.0,
 164400.0,
 337500.0,
 72500.0,
 143800.0,
 160000.0,
 118100.0,
 374000.0,
 202700.0,
 96600.0,
 65800.0,
 167400.0,
 179800.0,
 96700.0,
 266100.0,
 91100

In [57]:
y_predict

array([ 96017.47095359, 114069.24846599, 255995.36464501, ...,
       147737.15946261, 189936.75493775, 202961.94408164])

In [ ]:
#For Deployment

In [ ]:
run.wait_for_completion(show_output=False)  # specify True for a verbose log

In [ ]:
model = run.register_model(model_name='home',
                           model_path='outputs/home.pkl')
print(model.name, model.id, model.version, sep='\t')